The Cooking Chef Problem

Consider the case where the agent is your personal Chef.
In particular, the agent (the smiley on the map) wants to cook the eggs recipe according to your
indication (scrambled or pudding).
In order to cook the desired recipe, the agent must first collect the needed tools (the egg beater on the
map). Then he must reach the stove (the frying pan or the oven on the map). Finally, he can cook.
Not that there are two special interlinked cells (marked with the G) that allow the agent to go from
one side of the map to the other. But to do so, the agent needs to express his will to go on the other
side.
Cells in (4, 2) and (9, 3) are the special gate ones. They allow the agent to go from one side of the
map to another. Those two special cells are interlinked, but the agent needs to express his will to go
on the other side.
Since you are very hungry, it is fundamental that the agent cooks the eggs according to your taste
(scrambled/pudding) as fast as he can without letting you wait for more than necessary.
In order to apply optimal control techniques such as value iteration, you need to model the aforementioned scenario as an MDP. Recall that an MDP is defined as a tuple (S, A, P, R, γ), where:
S: The (finite) set of all possible states.
A: The (finite) set of all possible actions.
P: The transition function P : S × S × A → [0, 1], which maps (s
0
, s, a) to P(s
0
|s, a), i.e., the
probability of transitioning to state s
0 ∈ S when taking action a ∈ A in state s ∈ S. Note
that P
s
0∈S P(s
0
|s, a) = 1 for all s ∈ S, a ∈ A.
R: The reward function R : S × A × S → R, which maps (s, a, s0
) to R(s, a, s0
), i.e., the reward
obtained when taking action a ∈ A in state s ∈ S and arriving at state s
0 ∈ S.
γ: The discount factor which controls how important rewards are in the future.
In order to encode this problem as an MDP, you need to define each of the components of the tuple
for our particular problem. Note that there may be many different possible encodings (specify them
in the report).

Now, considering the problem as a model-free scenario, provide a program (written in Python,
possibly based on the labs) that can compute the optimal policy for this world by solely considering
the pudding eggs scenario. Draw the computed policy in the grid by putting the optimal action in
each cell. If multiple actions are possible, include the probability of each arrow. There may be
multiple optimal policies; pick one to show it. Note that the model is not available for computation
but must be encoded to be used as the "real-world" environment.

In [1]:
import numpy as np

# Define the grid world
grid_height = 5
grid_width = 10
start_state = (4, 3)
egg_beaters = [(1, 3), (8, 3)]
goals = [(1, 4), (8, 4)]
gate_cells = [(4, 2), (9, 3)]
walls = [(1, 1), (2, 1), (3, 1), (5, 1), (6, 1), (7, 1), (1, 2), (7, 2), (1, 3), (2, 3), (6, 3), (7, 3)]

# Define actions
actions = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # Left, Right, Up, Down

# Define parameters
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate

# Initialize Q-values
Q = np.zeros((grid_height, grid_width, len(actions)))

# Helper function to check if a state is valid
def is_valid_state(state):
    x, y = state
    return 0 <= x < grid_height and 0 <= y < grid_width and state not in walls

# Helper function to get the next state after taking an action
def get_next_state(state, action):
    next_state = (state[0] + action[0], state[1] + action[1])
    if is_valid_state(next_state):
        return next_state
    return state

# Reward function
def get_reward(state, action, next_state):
    reward = -1  # Default reward for each step
    if next_state in egg_beaters:
        reward += 10  # Additional reward for reaching egg beater
    if next_state in goals:
        reward += 100  # Additional reward for reaching goal
    return reward

# Q-learning algorithm
def q_learning():
    num_episodes = 1000
    for _ in range(num_episodes):
        state = start_state
        while state not in goals:
            action_index = choose_action(state)
            action = actions[action_index]
            next_state = get_next_state(state, action)
            reward = get_reward(state, action, next_state)
            next_action_max = np.max(Q[next_state[0], next_state[1]])
            Q[state[0], state[1], action_index] += alpha * (
                    reward + gamma * next_action_max - Q[state[0], state[1], action_index])
            state = next_state

# Helper function to choose an action based on epsilon-greedy policy
def choose_action(state):
    if np.random.rand() < epsilon:
        return np.random.choice(len(actions))
    else:
        return np.argmax(Q[state[0], state[1]])

# Compute optimal policy
q_learning()

# Print optimal policy
policy_symbols = ['←', '→', '↑', '↓']
policy = np.empty((grid_height, grid_width), dtype=str)
for i in range(grid_height):
    for j in range(grid_width):
        if (i, j) in walls:
            policy[i, j] = '█'  # Wall symbol
        elif (i, j) in gate_cells:
            policy[i, j] = 'G'  # Gate symbol
        elif (i, j) in egg_beaters:
            policy[i, j] = 'B'  # Egg beater symbol
        elif (i, j) in goals:
            policy[i, j] = 'F'  # Goal symbol
        else:
            best_action = np.argmax(Q[i, j])
            policy[i, j] = policy_symbols[best_action]

# Display the computed policy
for row in policy:
    print(' '.join(row))


↑ ← ← ↑ ↓ ↑ ← ← → ←
↓ █ █ █ F ← ← → → ←
← █ ↓ █ ↑ ↑ ← → → ←
↓ █ → → ↑ ← ← ↓ → ←
↓ → G ↑ ↑ ← ← → ← ←
